This is for a kaggle competition: https://www.kaggle.com/c/bosch-production-line-performance

In [ ]:
import xgboost
import pandas
import time
import io
import zipfile
import numpy
print('imports complete')

In [ ]:
# FYI: This takes about a minute and a half
def load_and_unzip(filename):
    print('loading and unzipping large training data file %s' % (filename))
    t0 = time.time()
    df = pandas.read_csv(filename, compression='zip')
    t1 = time.time()
    print('completed in %.3f seconds' % (t1-t0))
    return df

#train_numeric_filename = '../resources/train_numeric.csv.zip'
#df_explore = load_and_unzip(train_numeric_filename)
#test_numeric_filename = '../resources/test_numeric.csv.zip'
#df_test = load_and_unzip(test_numeric_filename)

Fill in the empty data with 0. get the mean and std of the entire training dataset, minus the id colum and the dependent variable column

In [ ]:
df_explore = pandas.read_csv('../resources/split_train_numeric_a.csv')
df_explore.fillna(0,inplace=True)
df_explore_meta = df_explore[df_explore.keys()[1:-1]].describe()
df_explore_meta

In [ ]:
def load_training_DMatrix(filename):
    arr = numpy.genfromtxt(filename, delimiter=',', skip_header=True, filling_values=0)
    return xgboost.DMatrix(arr[:,1:-1], label=arr[:,-1])
# load the testing file, without labels
def load_testing_DMatrix(filename):
    arr = numpy.genfromtxt(filename, delimiter=',', skip_header=True, filling_values=0)
    return arr[:,0], xgboost.DMatrix(arr[:,1:])

In [ ]:
dtrain = load_training_DMatrix('../resources/split_train_numeric_a.csv')
param = {'max_depth':2, 'eta':1, 'slient':1, 'objective':'binary:logistic'}
num_round = 2
bst = xgboost.train(param, dtrain, num_round)

In [ ]:
ids, dtest = load_testing_DMatrix('../resources/split_test_numeric_a.csv')

In [ ]:
pred = bst.predict(dtest)

In [ ]:
s= 0
for i, p in enumerate(pred):
    if p > 0.5:
        s = s + 1
print('%d,%d' %(ids[i], s))